In [1]:
import numpy as np
import torch
import time
import os

from sklearn import (linear_model, model_selection, preprocessing,
                     pipeline)
from scipy.spatial.distance import pdist

from kymatio.torch import HarmonicScattering3D

from kymatio.scattering3d.backend.torch_backend \
    import TorchBackend3D

from kymatio.scattering3d.utils \
    import generate_weighted_sum_of_gaussians

from kymatio.datasets import fetch_qm7
from kymatio.caching import get_cache_dir

import random

import pandas as pd
from ase.io import read

In [2]:
# Charger les données
train_energies = pd.read_csv('../data/energies/train.csv')
molecule_ids = train_energies['id'].values
energies = train_energies['energy'].values

# Initialiser les listes pour stocker toutes les positions et charges
all_positions = []
all_charges = []

# Lire tous les fichiers .xyz avec ASE
for mol_id in molecule_ids:
    xyz_path = f'../data/atoms/train/id_{mol_id}.xyz'
    atoms = read(xyz_path)
    
    # Obtenir les positions et numéros atomiques
    coords = atoms.get_positions()
    atomic_numbers = atoms.get_atomic_numbers()
    
    # Padding pour avoir une taille fixe
    max_atoms = 23  # Ajuster selon le nombre maximum d'atomes
    padded_coords = np.zeros((max_atoms, 3))
    padded_charges = np.zeros(max_atoms)
    
    n_atoms = len(coords)
    padded_coords[:n_atoms] = coords
    padded_charges[:n_atoms] = atomic_numbers
    
    all_positions.append(padded_coords)
    all_charges.append(padded_charges)

# Convertir en arrays numpy
pos = np.array(all_positions)
full_charges = np.array(all_charges)
target = energies

# Le reste du code reste identique à partir de la définition de n_molecules
n_molecules = len(pos)

# ...existing code...

In [ ]:
"""qm7 = fetch_qm7(align=True)
pos = qm7['positions']
full_charges = qm7['charges']

n_molecules = pos.shape[0]"""

In [ ]:
"""n_molecules_tot = len(pos)
n_molecules = 5
indices = random.sample(range(n_molecules_tot), n_molecules)

pos = pos[indices]
full_charges = full_charges[indices]"""

In [13]:
mask = full_charges <= 2
valence_charges = full_charges * mask

mask = np.logical_and(full_charges > 2, full_charges <= 10)
valence_charges += (full_charges - 2) * mask

mask = np.logical_and(full_charges > 10, full_charges <= 18)
valence_charges += (full_charges - 10) * mask

In [14]:
overlapping_precision = 1e-1
sigma = 2.0
min_dist = np.inf

for i in range(n_molecules):
    n_atoms = np.sum(full_charges[i] != 0)
    pos_i = pos[i, :n_atoms, :]
    min_dist = min(min_dist, pdist(pos_i).min())

delta = sigma * np.sqrt(-8 * np.log(overlapping_precision))
pos = pos * delta / min_dist

In [15]:
#M, N, O = 192, 128, 96
M, N, O = 32, 32, 32

grid = np.mgrid[-M//2:-M//2+M, -N//2:-N//2+N, -O//2:-O//2+O]
grid = np.fft.ifftshift(grid)

In [16]:
# J = 2
# L = 3
J = 1
L = 2
integral_powers = [0.5, 1.0, 2.0, 3.0]

scattering = HarmonicScattering3D(J=J, shape=(M, N, O),
                                  L=L, sigma_0=sigma,
                                  integral_powers=integral_powers)

In [17]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
scattering.to(device)

HarmonicScattering3D()

In [18]:
batch_size = 8
#batch_size = 16
n_batches = int(np.ceil(n_molecules / batch_size))

In [19]:
order_0, orders_1_and_2 = [], []
print('Computing solid harmonic scattering coefficients of '
      '{} molecules from the QM7 database on {}'.format(
        n_molecules,   "GPU" if use_cuda else "CPU"))
print('sigma: {}, L: {}, J: {}, integral powers: {}'.format(
        sigma, L, J, integral_powers))

this_time = None
last_time = None
for i in range(n_batches):
    this_time = time.time()
    if last_time is not None:
        dt = this_time - last_time
        print("Iteration {} ETA: [{:02}:{:02}:{:02}]".format(
                    i + 1, int(((n_batches - i - 1) * dt) // 3600),
                    int((((n_batches - i - 1) * dt) // 60) % 60),
                    int(((n_batches - i - 1) * dt) % 60)))
    else:
        print("Iteration {} ETA: {}".format(i + 1, '-'))
    last_time = this_time
    time.sleep(1)

    # Extract the current batch.
    start = i * batch_size
    end = min(start + batch_size, n_molecules)

    pos_batch = pos[start:end]
    full_batch = full_charges[start:end]
    val_batch = valence_charges[start:end]

    # Calculate the density map for the nuclear charges and transfer
    # to PyTorch.
    full_density_batch = generate_weighted_sum_of_gaussians(grid,
            pos_batch, full_batch, sigma)
    full_density_batch = torch.from_numpy(full_density_batch)
    full_density_batch = full_density_batch.to(device).float()

    # Compute zeroth-order, first-order, and second-order scattering
    # coefficients of the nuclear charges.
    full_order_0 = TorchBackend3D.compute_integrals(full_density_batch,
                                     integral_powers)
    full_scattering = scattering(full_density_batch)

    # Compute the map for valence charges.
    val_density_batch = generate_weighted_sum_of_gaussians(grid,
            pos_batch, val_batch, sigma)
    val_density_batch = torch.from_numpy(val_density_batch)
    val_density_batch = val_density_batch.to(device).float()

    # Compute scattering coefficients for the valence charges.
    val_order_0 = TorchBackend3D.compute_integrals(val_density_batch,
                                    integral_powers)
    val_scattering = scattering(val_density_batch)

    # Take the difference between nuclear and valence charges, then
    # compute the corresponding scattering coefficients.
    core_density_batch = full_density_batch - val_density_batch

    core_order_0 = TorchBackend3D.compute_integrals(core_density_batch,
                                     integral_powers)
    core_scattering = scattering(core_density_batch)

    # Stack the nuclear, valence, and core coefficients into arrays
    # and append them to the output.
    batch_order_0 = torch.stack(
        (full_order_0, val_order_0, core_order_0), dim=-1)
    batch_orders_1_and_2 = torch.stack(
        (full_scattering, val_scattering, core_scattering), dim=-1)

    order_0.append(batch_order_0)
    orders_1_and_2.append(batch_orders_1_and_2)

Computing solid harmonic scattering coefficients of 6591 molecules from the QM7 database on GPU
sigma: 2.0, L: 2, J: 1, integral powers: [0.5, 1.0, 2.0, 3.0]
Iteration 1 ETA: -
Iteration 2 ETA: [00:18:54]
Iteration 3 ETA: [00:17:31]
Iteration 4 ETA: [00:17:32]
Iteration 5 ETA: [00:17:28]
Iteration 6 ETA: [00:17:28]
Iteration 7 ETA: [00:17:24]
Iteration 8 ETA: [00:17:25]
Iteration 9 ETA: [00:17:24]
Iteration 10 ETA: [00:17:22]
Iteration 11 ETA: [00:17:24]
Iteration 12 ETA: [00:17:24]
Iteration 13 ETA: [00:17:19]
Iteration 14 ETA: [00:17:17]
Iteration 15 ETA: [00:17:19]
Iteration 16 ETA: [00:17:15]
Iteration 17 ETA: [00:17:12]
Iteration 18 ETA: [00:17:14]
Iteration 19 ETA: [00:17:11]
Iteration 20 ETA: [00:17:07]
Iteration 21 ETA: [00:17:10]
Iteration 22 ETA: [00:17:04]
Iteration 23 ETA: [00:17:05]
Iteration 24 ETA: [00:17:06]
Iteration 25 ETA: [00:17:05]
Iteration 26 ETA: [00:17:05]
Iteration 27 ETA: [00:17:03]
Iteration 28 ETA: [00:17:00]
Iteration 29 ETA: [00:16:58]
Iteration 30 ETA: [

In [20]:
order_0 = torch.cat(order_0, dim=0)
orders_1_and_2 = torch.cat(orders_1_and_2, dim=0)

order_0 = order_0.cpu().numpy()
orders_1_and_2 = orders_1_and_2.cpu().numpy()

In [21]:
order_0 = order_0.reshape((n_molecules, -1))
orders_1_and_2 = orders_1_and_2.reshape((n_molecules, -1))

In [22]:
"""basename = 'qm7_L_{}_J_{}_sigma_{}_MNO_{}_powers_{}.npy'.format(
        L, J, sigma, (M, N, O), integral_powers)"""
basename = 'traindataset_L_{}_J_{}_sigma_{}_MNO_{}_powers_{}.npy'.format(
        L, J, sigma, (M, N, O), integral_powers)

# cache_dir = get_cache_dir("qm7/experiments")
save_dir = "save"

filename = os.path.join(save_dir, 'order_0_' + basename)
np.save(filename, order_0)

filename = os.path.join(save_dir, 'orders_1_and_2' + basename)
np.save(filename, orders_1_and_2)

In [23]:
scattering_coef = np.concatenate([order_0, orders_1_and_2], axis=1)

In [ ]:
"""qm7 = fetch_qm7()
target = qm7['energies']"""

In [ ]:
# n_folds = 5
n_folds = 3

P = np.random.permutation(n_molecules).reshape((n_folds, -1))

cross_val_folds = []

for i_fold in range(n_folds):
    fold = (np.concatenate(P[np.arange(n_folds) != i_fold], axis=0),
            P[i_fold])
    cross_val_folds.append(fold)

In [27]:
alphas = 10.0 ** (-np.arange(1, 10))
for i, alpha in enumerate(alphas):
    scaler = preprocessing.StandardScaler()
    ridge = linear_model.Ridge(alpha=alpha)

    regressor = pipeline.make_pipeline(scaler, ridge)

    target_prediction = model_selection.cross_val_predict(regressor,
            X=scattering_coef, y=target, cv=cross_val_folds)

    MAE = np.mean(np.abs(target_prediction - target))
    RMSE = np.sqrt(np.mean((target_prediction - target) ** 2))

    print('Ridge regression, alpha: {}, MAE: {}, RMSE: {}'.format(
        alpha, MAE, RMSE))

Ridge regression, alpha: 0.1, MAE: 5.507477006220774, RMSE: 6.988796081317489
Ridge regression, alpha: 0.01, MAE: 5.345406858969104, RMSE: 6.832243762946825


/home/n7student/miniforge3/envs/env_tensorboard/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=3.91001e-09): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/n7student/miniforge3/envs/env_tensorboard/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=3.8376e-09): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/n7student/miniforge3/envs/env_tensorboard/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=4.21781e-09): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Ridge regression, alpha: 0.001, MAE: 5.284156123051306, RMSE: 6.8373216141688635
Ridge regression, alpha: 0.0001, MAE: 5.277232745983461, RMSE: 6.868797326387201
Ridge regression, alpha: 1e-05, MAE: 5.27243517496313, RMSE: 6.84576045460623
Ridge regression, alpha: 1e-06, MAE: 5.254580659655956, RMSE: 6.7957173526267765
Ridge regression, alpha: 1e-07, MAE: 5.235566034629851, RMSE: 6.753471198764981
Ridge regression, alpha: 1e-08, MAE: 5.233941947471445, RMSE: 6.749647209742278
Ridge regression, alpha: 1e-09, MAE: 5.2338631053661935, RMSE: 6.75398942074925


In [29]:
# Charger et préparer les données de test
test_positions = []
test_charges = []

# Lister tous les fichiers .xyz dans le dossier test
test_files = sorted(os.listdir('../data/atoms/test'))
test_ids = [int(f.split('_')[1].split('.')[0]) for f in test_files]  # Extraire les IDs

# Lire tous les fichiers .xyz de test
for xyz_file in test_files:
    xyz_path = os.path.join('../data/atoms/test', xyz_file)
    atoms = read(xyz_path)
    
    # Obtenir les positions et numéros atomiques
    coords = atoms.get_positions()
    atomic_numbers = atoms.get_atomic_numbers()
    
    # Padding pour avoir une taille fixe
    padded_coords = np.zeros((max_atoms, 3))
    padded_charges = np.zeros(max_atoms)
    
    n_atoms = len(coords)
    padded_coords[:n_atoms] = coords
    padded_charges[:n_atoms] = atomic_numbers
    
    test_positions.append(padded_coords)
    test_charges.append(padded_charges)

# Convertir en arrays numpy
test_pos = np.array(test_positions)
test_full_charges = np.array(test_charges)
n_test_molecules = len(test_pos)

# Appliquer la même normalisation que pour les données d'entraînement
test_pos = test_pos * delta / min_dist

# Calculer les coefficients de scattering pour les données de test
test_order_0 = []
test_orders_1_2 = []

for i in range(0, n_test_molecules, batch_size):
    end = min(i + batch_size, n_test_molecules)
    pos_batch = test_pos[i:end]
    charges_batch = test_full_charges[i:end]
    
    # Calculer les valence charges
    mask = charges_batch <= 2
    val_batch = charges_batch * mask
    mask = np.logical_and(charges_batch > 2, charges_batch <= 10)
    val_batch += (charges_batch - 2) * mask
    mask = np.logical_and(charges_batch > 10, charges_batch <= 18)
    val_batch += (charges_batch - 10) * mask
    
    # Calculer les descripteurs comme pour l'entraînement
    full_density = generate_weighted_sum_of_gaussians(grid, pos_batch, charges_batch, sigma)
    full_density = torch.from_numpy(full_density).to(device).float()
    
    val_density = generate_weighted_sum_of_gaussians(grid, pos_batch, val_batch, sigma)
    val_density = torch.from_numpy(val_density).to(device).float()
    
    core_density = full_density - val_density
    
    # Calculer les coefficients
    full_0 = TorchBackend3D.compute_integrals(full_density, integral_powers)
    full_s = scattering(full_density)
    
    val_0 = TorchBackend3D.compute_integrals(val_density, integral_powers)
    val_s = scattering(val_density)
    
    core_0 = TorchBackend3D.compute_integrals(core_density, integral_powers)
    core_s = scattering(core_density)
    
    # Empiler les coefficients
    batch_0 = torch.stack((full_0, val_0, core_0), dim=-1)
    batch_s = torch.stack((full_s, val_s, core_s), dim=-1)
    
    test_order_0.append(batch_0.cpu())
    test_orders_1_2.append(batch_s.cpu())

# Concatener tous les batches
test_order_0 = torch.cat(test_order_0, dim=0).numpy()
test_orders_1_2 = torch.cat(test_orders_1_2, dim=0).numpy()

# Reshape comme pour l'entraînement
test_order_0 = test_order_0.reshape((n_test_molecules, -1))
test_orders_1_2 = test_orders_1_2.reshape((n_test_molecules, -1))

# Concaténer les descripteurs
test_scattering_coef = np.concatenate([test_order_0, test_orders_1_2], axis=1)

# Entraîner le modèle final sur toutes les données d'entraînement
final_scaler = preprocessing.StandardScaler()
final_ridge = linear_model.Ridge(alpha=0.1)  # Utiliser le meilleur alpha trouvé
final_model = pipeline.make_pipeline(final_scaler, final_ridge)
final_model.fit(scattering_coef, target)

# Faire les prédictions sur les données de test
test_predictions = final_model.predict(test_scattering_coef)

# Créer et sauvegarder le fichier de prédictions
predictions_df = pd.DataFrame({
    'id': test_ids,
    'energy': test_predictions
})
predictions_df.to_csv('../data/energies/test_pred_scattering.csv', index=False)
print("Prédictions sauvegardées dans test_pred.csv")

Prédictions sauvegardées dans test_pred.csv


/home/n7student/miniforge3/envs/env_tensorboard/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=4.94015e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
